## 6.1 멀티 프로세스와 멀티 스레드

- 운영 체제에서 실행되는 각 프로그램 = 각각이 별도의 `프로세스`.
- 각 프로세스에는 하나 이상의 `스레드`가 있다.

`멀티 프로세스`: 별도의 메모리 영역을 가지며, 특별한 메커지늠으로만 통신할 수 있다. `프로세서`는 각 스레드에 대해 별도의 레지스터 집합을 불러오거나 저장, 프로세스간 데이터 공유와 통신용으로는 비효율적. 파이썬에서는 `subprocess` 사용

`멀티 스레드`: 단일 프로세스 내의 멀티 스레드는 동일한 메모리에 접근. 스레드는 데이터 공유를 통해 간단하게 통신한다. `threading` 모듈의 처리를 통해 한번에 한 스레드만 메모리 영역에 접근할 수 있다. 각 프로세스가 독립적인 `stack`, `heap`, `code`, `data` 를 가지는 반면, 한 프로세스에 속한 스레드는 `스택` 영역을 제외한 메모리 영역 공유.

- 멀티 스레드에서 한 스레드는 `인터프리터 락`을 획득하여야 실행될 수 있다.  인터프리터 락은 하나만 존재, 이것을 `GIL`이라고 함. GIL을 획득한 스레드가 어떤 자원을 사용시, 다른 스레드들은 이 자원에 접근할 수 없다.

`동시성`: 논리적으로 여러 작업이 동시에 실행되는 것처럼 보이는 것. 예를들어 I/O(파일 및 네트워크 소켓 입력 및 출력, 데이터를 주고받는 연산) 연산 등은 프로그램의 흐름에 큰 짐이 될 수 있다. 한 작업의 I/O 연산이 완료되기를 기다리는 동안 다른 작업을 수행하여 `유휴시간을 잘 활용`하는것이 `동시성`

`병렬성`: 물리적으로 여러 작업이 동시에 처리되는 것. 데이터 병렬성과 작업 병렬성으로 나뉨. 
`데이터 병렬성`: 같은 작업을 병렬처리 하는것. 
`작업 병렬성`: 서로 다른 작업을 병렬처리하는 것.
    - 웹 서버에서는 다수의 독립적인 요청을 병렬로 개별적으로 처리가능.

### 6.1.1 subprocess 모듈
- `subprocess` 모듈은 부모-자식 프로세스 쌍을 생성하는 데 사용됨. 
- 부모 프로세스는 사용자에 의해 실행
- 부모 프로세스는 차례로 다른 일을 처리하는 자식 프로세스의 인스턴스를 실행. 
- 자식 프로세스를 사용함으로써, 멀티 코어의 이점을 최대한 취하고, 동시성 문제를 운영 체제가 알아서 처리하도록 함

In [9]:
import subprocess
subprocess.run(["echo", "이것은 subprocess 입니다."])

CompletedProcess(args=['echo', '이것은 subprocess 입니다.'], returncode=0)

In [6]:
subprocess.run(["sleep", "10"]) #10초 동안 sleep

CompletedProcess(args=['sleep', '10'], returncode=0)

In [ ]:
#### 6.1.2 threading 모듈
- 스레드가 여러개로 분리되면, 스레드 간 데이터 공유의 복잡성이 증가.
- lock, deadklock을 회피하는데 주의를 기울여야 함.
- 파이썬에서는 단 하나의 메인 스레드만 존재
- 내부적으로 락을 관리하려면 `queue` 모듈 사용. 
- `스레드 데몬` 으로 변환하면, 워커 스레드가 작업완료시 종료되지 않고 계속 실행되는 문제 해결.
- queue.join()메서드는 큐가 빌때까지(큐의 모든 항목이 처리될 때까지_ 기다린다. 

In [27]:
import queue
import threading 

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()
        if item is None:
            break
        # 작업을 처리한다.
        print("스레드 {0}: 처리 완료 {1}".format(num+1, item))
        q.task_done()
        
if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i,))
        t.start()
        threads.append(t)
    
    for item in range(20):
        q.put(item)
        
    #모든 작업이 끝날 때까지 대기한다(block).
    q.join()
    
    # 워커 스레드를 종료한다(stop).
    for i in range(num_worker_threads):
        q.put(None)
    
    for t in threads:
        t.join()

스레드 1: 처리 완료 0스레드 4: 처리 완료 1스레드 5: 처리 완료 2스레드 2: 처리 완료 3스레드 3: 처리 완료 4


스레드 3: 처리 완료 5스레드 4: 처리 완료 6
스레드 5: 처리 완료 7

스레드 4: 처리 완료 8
스레드 4: 처리 완료 9
스레드 4: 처리 완료 10
스레드 4: 처리 완료 11
스레드 4: 처리 완료 12
스레드 4: 처리 완료 13
스레드 4: 처리 완료 14
스레드 4: 처리 완료 15
스레드 4: 처리 완료 16

스레드 4: 처리 완료 17
스레드 4: 처리 완료 18
스레드 4: 처리 완료 19

